In [1]:
import os
from dotenv import load_dotenv, find_dotenv 

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "ChatbotMemory"

In [2]:
from langchain_openai import OpenAIEmbeddings
openai_embedding = OpenAIEmbeddings()

In [3]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model = "gpt-3.5-turbo")

In [4]:
print(model.invoke("Hello").content)

Hello! How can I assist you today?


In [5]:
import bs4 
from langchain import hub

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import MessagesPlaceholder 

In [8]:
loader = WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",), bs_kwargs=dict(parse_only = bs4.SoupStrainer(class_ = ("post-content", "post-title", "post-header"))),)

In [9]:
doc = loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(doc)

In [14]:
vectorstore = Chroma.from_documents(documents=splits, embedding=openai_embedding)
retriever = vectorstore.as_retriever()

In [15]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000265BC3B7140>, search_kwargs={})

In [17]:
system_prompt = (
    "You are an assistant for question answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, Say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

In [18]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answering_chain = create_stuff_documents_chain(model, chat_prompt)

In [21]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [23]:
response = rag_chain.invoke({"input": "What is MRKL ?"})

In [25]:
response["answer"]

'MRKL, short for “Modular Reasoning, Knowledge and Language”, is a neuro-symbolic architecture for autonomous agents. It contains a collection of “expert” modules and a general-purpose LLM that routes inquiries to the best suitable expert module. These modules can be neural or symbolic in nature.'

In [27]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [28]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [32]:
history_aware_retriever = create_history_aware_retriever(model, retriever, contextualize_q_prompt)

In [33]:
from langchain.chains import create_retrieval_chain

In [35]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt), 
        MessagesPlaceholder("chat_history"), 
        ("human", "{input}")
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

In [37]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [38]:
from langchain_core.messages import HumanMessage, AIMessage

In [39]:
question1 = "What is Task Decomposition ?"

In [40]:
chat_history = []

In [41]:
message1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [43]:
message1["answer"]

'Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable. Techniques like Chain of Thought and Tree of Thoughts can help agents decompose hard tasks into multiple manageable tasks by instructing them to think step by step and explore multiple reasoning possibilities at each step. Task decomposition can be facilitated by using simple prompts, task-specific instructions, or human inputs to guide the agent in completing the overall task effectively.'

In [44]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [45]:
chat_history

[HumanMessage(content='What is Task Decomposition ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable. Techniques like Chain of Thought and Tree of Thoughts can help agents decompose hard tasks into multiple manageable tasks by instructing them to think step by step and explore multiple reasoning possibilities at each step. Task decomposition can be facilitated by using simple prompts, task-specific instructions, or human inputs to guide the agent in completing the overall task effectively.', additional_kwargs={}, response_metadata={})]

In [46]:
second_question = "What are the common ways of doing it ? "
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(message2["answer"])

Task decomposition can be done in common ways such as using simple prompts like "Steps for XYZ" or "What are the subgoals for achieving XYZ?" with Language Model (LLM) assistance, utilizing task-specific instructions tailored to the specific task at hand, or incorporating human inputs to guide the decomposition process effectively. These methods help break down complex tasks into smaller, more manageable steps, enabling agents to plan and execute tasks more efficiently.


In [47]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory 
from langchain_core.runnables.history import RunnableWithMessageHistory

In [48]:
store = {}

In [49]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() 
    return store[session_id]

In [50]:
converstational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input", 
    history_messages_key="chat_history", 
    output_messages_key="answer",
)

In [51]:
converstational_rag_chain.invoke(
    {"input": "What is Task Decompostion ?"}, 
    config={
        "configurable": {"session_id": "abc123"}
    }
)["answer"]

'Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an autonomous agent or AI system. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable subtasks by guiding the model to "think step by step" or explore multiple reasoning possibilities at each step. This process allows the agent to plan ahead and execute tasks more effectively.'

In [52]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decompostion ?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an autonomous agent or AI system. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable subtasks by guiding the model to "think step by step" or explore multiple reasoning possibilities at each step. This process allows the agent to plan ahead and execute tasks more effectively.', additional_kwargs={}, response_metadata={})])}

In [53]:
converstational_rag_chain.invoke(
    {"input": "What are the common ways of doing it ?"},
    config={"configurable": {"session_id": "abc123"}}
)["answer"]

'Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting like "Steps for XYZ" or asking for subgoals, providing task-specific instructions like "Write a story outline" for a novel-writing task, or incorporating human inputs to guide the decomposition process. These approaches help in breaking down complex tasks into smaller components that can be easily tackled and executed by an autonomous agent or AI system.'

In [55]:
for i in store["abc123"].messages:
    if isinstance(i, AIMessage):
        prefix = "AI"
    else:
        prefix = "user"
    
    print(f"{prefix} : {i.content}\n")

user : What is Task Decompostion ?

AI : Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an autonomous agent or AI system. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable subtasks by guiding the model to "think step by step" or explore multiple reasoning possibilities at each step. This process allows the agent to plan ahead and execute tasks more effectively.

user : What are the common ways of doing it ?

AI : Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting like "Steps for XYZ" or asking for subgoals, providing task-specific instructions like "Write a story outline" for a novel-writing task, or incorporating human inputs to guide the decomposition process. These approaches help in breaking down complex tasks into smaller components that can be easily tackled and executed by an aut

In [56]:
converstational_rag_chain.invoke(
    {"input": "What is a prompt technique like step xyz?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'A prompt technique like Chain of Thought (CoT) involves guiding a model to "think step by step" in order to decompose complex tasks into smaller and more manageable steps. This technique helps enhance model performance on difficult tasks by utilizing more test-time computation and shedding light on the model\'s thinking process. CoT transforms big tasks into multiple manageable tasks, making it easier for the agent to plan and execute the necessary actions.'

In [57]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decompostion ?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an autonomous agent or AI system. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable subtasks by guiding the model to "think step by step" or explore multiple reasoning possibilities at each step. This process allows the agent to plan ahead and execute tasks more effectively.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the common ways of doing it ?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting like "Steps for XYZ" or asking for subgoals, providing task-specific instructions lik

In [58]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decompostion ?

AI: Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable for an autonomous agent or AI system. Techniques like Chain of Thought (CoT) and Tree of Thoughts help in decomposing hard tasks into multiple manageable subtasks by guiding the model to "think step by step" or explore multiple reasoning possibilities at each step. This process allows the agent to plan ahead and execute tasks more effectively.

User: What are the common ways of doing it ?

AI: Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting like "Steps for XYZ" or asking for subgoals, providing task-specific instructions like "Write a story outline" for a novel-writing task, or incorporating human inputs to guide the decomposition process. These approaches help in breaking down complex tasks into smaller components that can be easily tackled and executed by an autonom